# Concurrency Models in Python

## Spinner with Threads

In [1]:
import itertools
import time
from threading import Thread, Event

def spin(msg: str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

The `done` argument is an instance of `threading.Event`, a simple object to synchronize threads. 

The `\r` means **Carriage Return**. 

The `Event.wait(timeout=None)` method returns `True` when the event is set by another thread; if the `timeout` elapses, it returns `False`. 

In [2]:
def supervisor() -> int:
    done = Event()
# create a new `Thread`, provide a function as the `target` keyword and positional arguments to the `target` as a tuple passed via `args`
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')     
    spinner.start()
    result = slow() # call `slow`, which blocks the `main` thread. Meanwhile, the secondary thread is running the spinner animation
    done.set()      # set the `Event` flag to `True`; this will terminate the `for` loop inside the `spin` function
    spinner.join()  # wait until the `spinner` thread finishes
    return result

In [3]:
print('Answer:', supervisor())

spinner object: <Thread(Thread-5 (spin), initial)>
Answer: 42  


The `threading.Event` instance is the key to coordinate the activities of the `main` thread and the `spinner` thread. 

## Spinner with Processes

In [4]:
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize


def spin(msg: str, done: synchronize.Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def slow() -> int:
    time.sleep(3)
    return 42


def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))    # Basic usage of the `Process` class is similar to `Thread`
    print(f'spinner object: {spinner}')     
    spinner.start()
    result = slow() 
    done.set()      
    spinner.join()  
    return result

In [5]:
print('Answer:', supervisor())

spinner object: <Process name='Process-1' parent=4476 initial>
Answer: 42


## Spinner with Coroutines

### In Contrast

#### [with `await`: 18](./with_await.py:18)

#### [without `await`: 18](./without_await.py:18)
